In [1]:
import pandas as pd

In [2]:
PATH = '/home/ivan/test_jun/'

In [3]:
clust = pd.read_csv(PATH + 'address_clust.csv')

In [4]:
clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [5]:
clust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6996 entries, 0 to 6995
Data columns (total 2 columns):
address_id    6996 non-null int64
cluster_id    6996 non-null int64
dtypes: int64(2)
memory usage: 109.4 KB


In [6]:
stats = pd.read_csv(PATH + 'address_stats.csv')

In [7]:
stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [8]:
stats = stats.merge(clust, how='left', on='address_id').fillna(0)
stats.drop_duplicates(inplace=True)

In [9]:
stats['cluster_id'] = stats['cluster_id'].astype(int)

In [10]:
stats['cluster_id'].value_counts()

1    24195
0    16820
2    13430
Name: cluster_id, dtype: int64

In [11]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    return (np.abs(array - value)).argmin()

In [12]:
stats['to_cluster_0'] = 0
stats['to_cluster_1'] = 0
stats['to_cluster_2'] = 0

In [13]:
for name, group in stats.groupby('transaction_id'):
    group.sort_values(by='sent', ascending=False)
    send = [group[group['sent'] > 0]['sent'].values, group[group['sent'] > 0]['id'].values]
    receive = [group[group['received'] > 0]['received'].values, group[group['received'] > 0]['cluster_id'].values]
    last_cluster = -1
    while True:
        for id, item in enumerate(send[0]):
            idx = find_nearest(receive[0], item)
            cluster = receive[1][idx]
            was_receive = receive[0][idx]
            receive[0][idx] = receive[0][idx] - item
            if receive[0][idx] <= 0:
                receive[0] = np.delete(receive[0], idx)
                receive[1] = np.delete(receive[1], idx)
            if (item - was_receive) > 0:
                send[0][id] = send[0][id] - was_receive
                stats.loc[(stats['transaction_id'] == name) & (stats['id'] == send[1][id]), f'to_cluster_{cluster}'] += was_receive
                break
            else:
                stats.loc[(stats['transaction_id'] == name) & (stats['id'] == send[1][id]), f'to_cluster_{cluster}'] += item
                send[0] = np.delete(send[0], id)
                send[1] = np.delete(send[1], id)   
                break
        if len(send[0]) == 0:
            break
        if len(receive[0]) == 0:
            break

In [14]:
stats.head()

,id,address_id,transaction_id,received,sent,cluster_id,to_cluster_0,to_cluster_1,to_cluster_2
0,87134765,46402336,19162323,0.0,1800000.0,0,1800000.0,0.0,0.0
1,87134766,45919235,19162323,0.0,1071052.0,0,21052.0,1000000.0,0.0
2,87134767,46529090,19162323,1000000.0,0.0,1,0.0,0.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0,0,0.0,0.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0,1,990000.0,0.0,0.0


In [15]:
def how_many_btc(c_from, c_to):
    btc = stats[(stats['sent'] > 0) & (stats['cluster_id'] == c_from)][f'to_cluster_{c_to}'].sum()/10**8
    print(f'{btc} btc отправил кластер {c_from} в кластер {c_to}')

### Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?

In [16]:
how_many_btc(1, 2)

64.62162 btc отправил кластер 1 в кластер 2


In [17]:
how_many_btc(2, 1)

67.91 btc отправил кластер 2 в кластер 1


### Сколько биткоинов отправил кластер 1 в кластер 0 и сколько кластер 0 отправил в кластер 1?

In [18]:
how_many_btc(1, 0)

291.5108346 btc отправил кластер 1 в кластер 0


In [19]:
how_many_btc(0, 1)

288.7905886 btc отправил кластер 0 в кластер 1


### Сколько биткоинов отправил кластер 2 в кластер 0 и сколько кластер 0 отправил в кластер 2?

In [20]:
how_many_btc(2, 0)

2.43507 btc отправил кластер 2 в кластер 0


In [21]:
how_many_btc(0, 2)

6.09640861 btc отправил кластер 0 в кластер 2


### Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

In [29]:
(stats[stats['cluster_id'] == 1]['sent'].sum() - stats[stats['cluster_id'] == 1]['to_cluster_0'].sum() - stats[stats['cluster_id'] == 1]['to_cluster_1'].sum() - stats[stats['cluster_id'] == 1]['to_cluster_2'].sum())/10**8, (stats[stats['cluster_id'] == 2]['sent'].sum() - stats[stats['cluster_id'] == 2]['to_cluster_0'].sum() - stats[stats['cluster_id'] == 2]['to_cluster_1'].sum() - stats[stats['cluster_id'] == 2]['to_cluster_2'].sum())/10**8

(0.502344, 0.37295861)

В итоге получился не особо точный на мой взгляд алгоритм, смущают крошечные комиссии

Основная проблема - неизвестно в какой доле разделяется сумма транзакциий. Например, если с двух адресов средства поступают на два дугих адреса, то они могут быть распределены вообще в любой пропорции, и я не понял как это побороть.  
В итоге, просто смотрел какая отправленная сумма наиболее близка к принятой сумме, и считал, что средства отправлялись именно на этот адрес.  
В этом подходе есть большой минус: если входящая сумма меньше, чем исходящая, то считается, что вся отправленная сумма ушла туда, и теряется информация о комиссии.  
А в целом, вроде осилил) Спасибо!